In [1]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# from sklearn.metrics import confusion_matrix, classification_report
from xgboost import XGBClassifier
from classifiers import Classifiers

import matplotlib.pyplot as plt

from nltk.corpus import stopwords

from utils import create_pipe, run_classifiers, get_conf_matrix

In [2]:
path="../data/combined.pickle"

try:
    with open(path,'rb') as handle:
        pickleload=pickle.load(handle)
except FileNotFoundError as e:
    e.strerror = "Pls run 01_scrape_reddit first to pull the data and 02_EDA to merge data."
    raise e

df=pd.DataFrame(pickleload)

df

,post,label,post2
0,this is the nail in the coffin for the idea of...,0,this is the nail in the coffin for the idea of...
1,"i’m closeted, always been, and always will be....",0,"i’m closeted, always been, and always will be...."
2,the fifa world cup in qatar should be a remind...,0,the fifa world cup in qatar should be a remind...
3,we moved from the dc metro area last year to t...,0,we moved from the dc metro area last year to t...
4,they spend so much time focusing on arbitrary ...,0,they spend so much time focusing on arbitrary ...
...,...,...,...
10078,"if jesus died for our sins, what's keeping us ...",1,"if jesus died for our sins, what's keeping u f..."
10079,hello everybody it may seem like a dumb questi...,1,hello everybody it may seem like a dumb questi...
10080,today's readings: 1 corinthians 1:4-8 &gt;i ...,1,today's readings: 1 corinthian 1:4-8 &gt;i tha...
10081,i don't propose this question in the sense of ...,1,i don't propose this question in the sense of ...


### Prepare the dataset

In [3]:
X=df['post']
y=df['label']

## Add Random Forest and XGBoost into the mix

In [4]:
classifiers_list=[
    {
        'cls':MultinomialNB(),
        'name':'NaiveBayes',
        'fixed_params':{'min_df': 0.05, 'max_features': 3500, 'max_df': 0.8,'use_idf': False, 'ngram_range': (1, 2)},
    },
    {
        'cls':RandomForestClassifier(),
        'name':'RandomForest',
        'fixed_params':{'min_df': 0.05, 'max_features': 3500, 'max_df': 0.8,'use_idf': False, 'ngram_range': (1, 2)},
        'float_params':{
            'cls__max_depth':[60,80,100,120,140],
            'cls__min_samples_split':[3,4,5,6,7],
            'cls__min_samples_leaf':[2,3,4]
        }
    },
    {
        'cls':XGBClassifier(),
        'name':'XGBoost',
        'fixed_params':{'min_df': 0.05, 'max_features': 3500, 'max_df': 0.8,'use_idf': False, 'ngram_range': (1, 2)},
        'float_params':{
            'cls__eta':[0.08,0.1,0.12],
            'cls__gamma':[1,10,20,50],
            'cls__max_depth':[6,7,8],
            'cls__subsample_bytree':[0.5,0.7,0.9]
        }
    }
]
run_classifiers(classifiers_list,X,y,tfidf=True)

======= Running classifier: NaiveBayes =======
ROC AUC with CV=5: 0.825646048026959
======= Running classifier: RandomForest =======
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best parameters and accuracy
{'cls__min_samples_split': 7, 'cls__min_samples_leaf': 4, 'cls__max_depth': 60}
ROC AUC with CV=5: 0.834905267794349
======= Running classifier: XGBoost =======
Fitting 5 folds for each of 30 candidates, totalling 150 fits
[19:51:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "subsample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Best parameters and accuracy
{'cls__subsample_bytree': 0.9, 'cls__max_depth': 6, 'cls__gamma': 1, 'cls__eta': 0.12}
ROC AUC with

In [5]:
for classifier in classifiers_list:
    name=classifier['name']
    params={}
    if 'params' in classifier:
        kw={}
        params=classifier['params']
    cv=GridSearchCV(create_pipe(classifier['cls']),
                     params,cv=5,n_jobs=-1)
    cv.fit(X,y)
    display(cv.best_params_)
    display(cv.best_score_)

TypeError: create_pipe() missing 1 required positional argument: 'tfidf'

### Let's check out the top features as processed by `TfidfVectorizer`